In [ ]:
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import regnet_y_400mf
from torchvision.transforms import v2

from src.train_model import train_one_epoch

### 1. Загрузка и предобразботка данных

In [ ]:
train_transforms = v2.Compose([
    # v2.RandomRotation([-5, 5], fill=255),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.RandomPhotometricDistort(contrast=[0.9, 1.1],
                                hue=[-0.05, 0.05]),
                                v2.Resize((224, 224)),
                                v2.ToTensor(),
                                # v2.Normalize([0.5], [0.5])
                                ])
val_transforms = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToTensor(),
    # v2.Normalize([0.5], [0.5])
    ])

In [ ]:
train_dataset = ImageFolder('ogyeiv2/train', transform=train_transforms)
val_dataset = ImageFolder('ogyeiv2/test', transform=val_transforms)

In [ ]:
classes = train_dataset.classes

In [ ]:
fig = plt.figure(figsize=(25, 5))
for i in range(1, 6):
    image, label = train_dataset[100 + i]
    plt.subplot(1, 5, i)
    plt.imshow(image.permute((1, 2, 0)))
    print(classes[label])
plt.show()


In [ ]:
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print('Количество классов:', len(classes))
print('Количество изображений в обучающем датасете:', len(train_dataset))
print('Количество изображений в валидационном датасете:', len(val_dataset))

### 2. Объявление модели

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Устройство обучения:', device)

In [ ]:
model = regnet_y_400mf(weights='IMAGENET1K_V2')

In [ ]:
model.fc = nn.Linear(in_features=440, out_features=84)

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
for param in model.fc.parameters():
    param.requires_grad = True

In [ ]:
model.to(device)

### 3. Обучение или дообучение

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
EPOCHS = 10

In [ ]:
train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    model.train()
    train_loss = train_one_epoch(train_loader=train_loader, model=model,
                                 criterion=criterion, optimizer=optimizer,
                                 device=device, epoch_index=epoch)
    print(f'Epoch: {epoch}, train loss: {train_loss}')
    train_losses.append(train_loss)
    
    running_vloss = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_vloss += loss.item()
    val_loss = running_vloss / len(val_loader)
    print(f'Epoch: {epoch}, val loss: {val_loss}')
    val_losses.append(val_loss)
    

### 4. Оценка качества